* LogReg 
    > Accuracy = 0.84
    > 
    > ROC-AUC = 0.88409
* SVC
    > Accuracy = 0.87
    > 
    > ROC-AUC = 0.93501
* Neural Network
    > Accuracy = 0.875
    >
    > ROC-AUC = 0.9507035
* Boosting (catBoost)
    > Accuracy = 0.915
    >
    > ROC-AUC = 0.96335

In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import catboost
import lightgbm as lgb

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.mixture import BayesianGaussianMixture
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score

In [31]:
df_train = pd.read_csv('./data/train.csv', header=None)
df_y = pd.read_csv('./data/trainLabels.csv', header=None).values
df_test = pd.read_csv('./data/test.csv', header=None)
df_sub = pd.DataFrame({
    'Id' : range(1, len(df_test)+1),
    'Solution' : 0,
})

In [32]:
scaler = StandardScaler()
df_train = scaler.fit_transform(df_train.values)
df_test = scaler.fit_transform(df_test.values)
df_train = pd.DataFrame(df_train)
df_test = pd.DataFrame(df_test)

In [5]:
# X_train, X_test, y_train, y_test = train_test_split(df_train, df_y, test_size=0.2, random_state=0)
# y_train = y_train.ravel()
# y_test = y_test.ravel()

#### Gaussian Model

In [33]:
x_all = pd.concat([df_train, df_test])

In [34]:
gmm = BayesianGaussianMixture(n_components=7, n_init=3)
gmm.fit(x_all)
X_train = gmm.predict_proba(df_train)
X_test = gmm.predict_proba(df_test)

#### LogReg

In [ ]:
clf = LogisticRegression()
clf.fit(X_train, y_train)
# y_pred = clf.predict(X_test)
# y_score = clf.predict_proba(X_test)

# clf.fit(df_train, df_y.ravel())

#### SVM

In [ ]:
clf = SVC(probability=True)
# clf.fit(X_train, y_train)
# y_pred = clf.predict(X_test)
# y_score = clf.predict_proba(X_test)

clf.fit(df_train, df_y.ravel())

#### Neural Network

In [336]:
# X_train_t = torch.tensor(X_train, dtype=torch.float32)
# X_test_t = torch.tensor(X_test, dtype=torch.float32)
# y_train_t = torch.tensor(y_train, dtype=torch.long)
# y_test_t = torch.tensor(y_test, dtype=torch.long)

X_train_t = torch.tensor(df_train, dtype=torch.float32)
X_test_t = torch.tensor(df_test, dtype=torch.float32)
y_train_t = torch.tensor(df_y.ravel(), dtype=torch.long)

In [339]:
dataset = TensorDataset(X_train_t, y_train_t)
train_loader = DataLoader(dataset, shuffle=True)

In [340]:
class clf(nn.Module):
    def __init__(self):
        super(clf, self).__init__()
        self.fc1 = nn.Linear(in_features=40, out_features=25)
        self.fc2 = nn.Linear(in_features=25, out_features=10)
        pass
        
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu6(x)
        x = self.fc2(x)
        return x

model = clf()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
# scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

In [342]:
model.train()
history_loss = []
num_epoch = 10
for epoch in range(num_epoch):
    for x, y in train_loader:
        optimizer.zero_grad()
        y_pred = model(x)
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()
    # scheduler.step()
    _loss = loss.detach().numpy().tolist()
    history_loss.append(_loss)
    print(f'Epoch: {epoch+1},  loss = {round(_loss, 5)}')
    # print(f'Epoch: {epoch+1},  lr = {round(scheduler.get_last_lr()[0], 5)},  loss = {round(_loss, 5)}')

print('----------------------')
with torch.no_grad():
    model.eval()
    y_pred = model(X_test_t)
    probabilities = F.softmax(y_pred, dim=1)
    _, predicted = torch.max(probabilities, 1)
predicted_classes = predicted.numpy()
# print(f'Accuracy = {accuracy_score(y_test, predicted_classes)}')
# print(f'ROC-AUC = {roc_auc_score(y_test, probabilities[:, 1].numpy())}')

Epoch: 1,  loss = 0.11654
Epoch: 2,  loss = 0.0
Epoch: 3,  loss = 0.15109
Epoch: 4,  loss = 0.20077
Epoch: 5,  loss = 0.00222
Epoch: 6,  loss = 0.09346
Epoch: 7,  loss = 0.01554
Epoch: 8,  loss = 0.07233
Epoch: 9,  loss = 1e-05
Epoch: 10,  loss = 0.0
----------------------


#### CatBoo

#### LightG

In [38]:
clf = lgb.LGBMClassifier(objective='binary',  metric='accuracy', n_jobs=-1)
# clf.fit(X_train, y_train)
# y_pred = clf.predict(X_test)
# y_score = clf.predict_proba(X_test)

# clf.fit(df_train, df_y.ravel())

[LightGBM] [Info] Number of positive: 510, number of negative: 490
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000936 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 150
[LightGBM] [Info] Number of data points in the train set: 1000, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.510000 -> initscore=0.040005
[LightGBM] [Info] Start training from score 0.040005
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


ValueError: For early stopping, at least one dataset and eval metric is required for evaluation

#### evalution model

In [21]:
print(f'Accuracy = {accuracy_score(y_test, y_pred)}')
print(f'ROC-AUC = {roc_auc_score(y_test, y_score[:, 1])}')

Accuracy = 0.88
ROC-AUC = 0.9530316833687621


#### Predict and save data

In [36]:
y_pred = clf.predict(X_test)
# y_pred = predicted_classes
df_sub['Solution'] = y_pred

In [37]:
df_sub.to_csv('london.csv', index=False)